In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Flatten, Dense, Dropout
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras import backend as K

K.clear_session()

img_width, img_height =  176, 208

### Build the network 
img_input = Input(shape=(img_width, img_height, 1))
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

model = Model(input = img_input, output = x)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 176, 208, 1)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 176, 208, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 176, 208, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 88, 104, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 88, 104, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 88, 104, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 44, 52, 128)       0         
Total para

/home/avell/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`


In [9]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
[layer.name for layer in model.layers]

['input_1',
 'block1_conv1',
 'block1_conv2',
 'block1_pool',
 'block2_conv1',
 'block2_conv2',
 'block2_pool']

In [10]:
import h5py
weights_path = '/home/avell/Downloads/vgg19_weights.h5' # ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5)
f = h5py.File(weights_path)
list(f.keys())

['block1_conv1',
 'block1_conv2',
 'block1_pool',
 'block2_conv1',
 'block2_conv2',
 'block2_pool',
 'block3_conv1',
 'block3_conv2',
 'block3_conv3',
 'block3_conv4',
 'block3_pool',
 'block4_conv1',
 'block4_conv2',
 'block4_conv3',
 'block4_conv4',
 'block4_pool',
 'block5_conv1',
 'block5_conv2',
 'block5_conv3',
 'block5_conv4',
 'block5_pool',
 'fc1',
 'fc2',
 'flatten',
 'predictions']

In [11]:
# list all the layer names which are in the model.
layer_names = [layer.name for layer in model.layers]

f["block1_conv1"].attrs["weight_names"]
# """
# # Here we are extracting model_weights for each and every layer from the .h5 file

# >>> f["model_weights"]["block1_conv1"].attrs["weight_names"]
# array([b'block1_conv1/kernel:0', b'block1_conv1/bias:0'], 
#       dtype='|S21')
# # we are assiging this array to weight_names below 

# >>> f["model_weights"]["block1_conv1"]["block1_conv1/kernel:0]
# <HDF5 dataset "kernel:0": shape (3, 3, 3, 64), type "<f4">
# # The list comprehension (weights) stores these two weights and bias of both the layers 

# >>>layer_names.index("block1_conv1")
# 1

# >>> model.layers[1].set_weights(weights)
# # This will set the weights for that particular layer.
# With a for loop we can set_weights for the entire network.
# """

array([b'block1_conv1_W_1:0', b'block1_conv1_b_1:0'], 
      dtype='|S18')

In [12]:
for i in layer_dict.keys():
    if i == 'input_1':
        break
    weight_names = f[i].attrs["weight_names"]
    weights = [f[i][j] for j in weight_names]
    index = layer_names.index(i)
    model.layers[index].set_weights(weights)
model.layers

In [13]:
import numpy as np
import pandas as pd
import itertools
import glob
import subprocess
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

def call_shell(cmd, *argv):
    list_ = [cmd];    [list_.append(arg) for arg in argv]
    out = subprocess.run(list_, stdout=subprocess.PIPE)
    return out.stdout.decode('utf-8')

DEST = '/media/avell/Novo volume/CV_SARGITAL/'

def features_extraction(fold):
    reshape_im = lambda im: np.expand_dims(np.reshape(im,(im.shape[0],im.shape[1],1)),axis=0)
    reshape = lambda array: [np.array([i.ravel(),]) for i in array ]

    location_0 = fold+'0.0/'
    location_1 = fold+'1.0/'
    
    files_0 = call_shell('ls',location_0).split('\n')[:-1]
    files_1 = call_shell('ls',location_1).split('\n')[:-1]

    features_1 = []
    features_0 = []
    
    for i in files_0:
        im_0 = imread(location_0+i)
        im_0 = reshape_im(im_0)
        features_0.append(model.predict(im_0))    
    for i in files_1:
        im_1 = imread(location_1+i)
        im_1 = reshape_im(im_1)
        features_1.append(model.predict(im_1))
        
    mt_f0 = reshape(features_0)
    mt_f1 = reshape(features_1)

    mt_f0 = np.array(mt_f0).reshape(len(mt_f0),mt_f0[0].shape[1])
    mt_f1 = np.array(mt_f1).reshape(len(mt_f1),mt_f1[0].shape[1])

    mt_f0 = np.hstack([mt_f0 ,np.zeros([mt_f0.shape[0],1])])
    mt_f1 = np.hstack([mt_f1 ,np.ones([mt_f1.shape[0],1])])

    df = pd.DataFrame(np.vstack([mt_f0,mt_f1]))
    
    return df
        
## collect these features and create a dataframe and train a classfier on top of it.

In [14]:
k_features_train = []
k_features_test = []

for i in range(10):
    fold_tr = DEST+'fold_{}/train/'.format(i)
    fold_te = DEST+'fold_{}/test/'.format(i)
    k_features_train.append(features_extraction(fold_tr))
    k_features_test.append(features_extraction(fold_te))

In [15]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score,confusion_matrix, classification_report
import csv

def meanMetrics(predList, yList):
    PR, RC, F1, ACC, SP = [],[],[],[],[]
    for p, y in zip(predList, yList):
        pr,rl,f1,sp = precision_recall_fscore_support(p, y)
        acc = accuracy_score(p, y)
        tn, fp, fn, tp = confusion_matrix(p, y).ravel()
        specificity = 1.0*tn / (tn+fp)
        PR.append(pr); RC.append(rl); F1.append(f1);
        ACC.append(acc); SP.append(specificity)
    PR, RC, F1, ACC, SP = np.array(PR),np.array(RC),np.array(F1),np.array(ACC),np.array(SP)
    print('Mean\t| Precision |\t| Recall |\t| F-score |\t| Accuracy |\t| Specificity |')
    exit = '| {} || {} || {} || {} || {} |'
    print(exit.format(np.mean(PR),np.mean(RC),np.mean(F1),np.mean(ACC),np.mean(SP)))


axialPred, y_labels = [],[]

for i in range(10):
    c_y = k_features_test[0].shape[1]
    x_test = k_features_test[i].drop(c_y-1,axis=1)
    x_train = k_features_train[i].drop(c_y-1,axis=1)
    y_test = k_features_test[i][c_y-1]
    y_train = k_features_train[i][c_y-1]
    
    svm = SVC(C=100,gamma=2e-6)
    svm.fit(x_train,y_train)
    pred = svm.predict(x_test)
    
    fold = DEST+'fold_{}'.format(i)
    
    pr,rl,f1,sp = precision_recall_fscore_support(pred, y_test)
    acc = accuracy_score(pred, y_test)
    tn, fp, fn, tp = confusion_matrix(pred, y_test).ravel()
    specificity = 1.0*tn / (tn+fp+1e-16)
    
    with open(fold+'/fold_{}_metrics.csv'.format(i),'w') as csvFile:
        filewriter = csv.writer(csvFile)
        filewriter.writerow(['class','precision','recall','fscore','support','accuracy','specificity'])
        filewriter.writerow([0,pr[0],rl[0],f1[0],sp[0],'-','-'])
        filewriter.writerow([1,pr[1],rl[1],f1[1],sp[1],'-','-'])
        filewriter.writerow(['-','-','-','-','-',acc,specificity])
    csvFile.close()
    
    np.save(fold+'/fold_{}_pred_labels.npy'.format(i),pred)
    
    axialPred.append(pred)
    y_labels.append(y_test)

meanMetrics(axialPred,y_labels)

Mean	| Precision |	| Recall |	| F-score |	| Accuracy |	| Specificity |
| 0.6899658674658674 || 0.6839664502164502 || 0.6721114588960193 || 0.6789473684210525 || 0.6231818181818183 |


In [ ]:
DEST = '/media/avell/Novo volume/CV_AXIAL/'
EXT = '_a.jpg'


for i in range(10):
    ldst = DEST+'fold_{}/train/'.format(i)
    k_labels = []
    for j in range(10):
        fold = ldst+'fold_{}'.format(j)

        fold_tr = fold+'/train/'.format(i)
        fold_te = fold+'/test/'.format(i)
        print(fold_tr)
        k_features_train = features_extraction(fold_tr)
        k_features_test = features_extraction(fold_te)

        c_y = k_features_test.shape[1]
        x_test = k_features_test.drop(c_y-1,axis=1)
        x_train = k_features_train.drop(c_y-1,axis=1)
        y_test = k_features_test[c_y-1]
        y_train = k_features_train[c_y-1]

        svm = SVC()
        svm.fit(x_train,y_train)
        pred = svm.predict(x_test)

        k_labels.append(pred.ravel())
    np.save(ldst+'features_'+'fold_{}'.format(i),np.array(k_labels))